In [1]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
browser = webdriver.Chrome("C:/Users/LOVE.DESKTOP-5CS2DDB/anaconda3/chromedriver_win32/chromedriver.exe", options=options)
url = "https://www.imdb.com/title/tt10919420/reviews?ref_=tt_urv"
browser.get("https://www.imdb.com/title/tt10919420/reviews?ref_=tt_urv")


import time
interval = 2 # 2초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = browser.execute_script("return document.body.scrollHeight")

# 반복 수행

while True:
    # 스크롤을 가장 아래로 내림
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    browser.find_element_by_class_name("ipl-load-more__button").click()
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = browser.execute_script("return document.body.scrollHeight")
    print(prev_height)
    print(curr_height)

    if curr_height > 130000:
        break
#     if curr_height == prev_height:
    
#      prev_height = curr_height



print("스크롤 완료")

7163
13214
7163
19850
7163
26168
7163
32514
7163
38947
7163
44867
7163
51461
7163
57513
7163
64378
7163
70679
7163
76967
7163
83414
7163
89645
7163
96443
7163
102277
7163
108558
7163
114946
7163
121393
7163
127368
7163
133166
스크롤 완료


In [2]:
import csv
import requests
from bs4 import BeautifulSoup



headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
res = requests.get(url, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

In [6]:


soup = BeautifulSoup(browser.page_source, "lxml")
reviews = soup.find_all("a", attrs={"class":["title"]})

filename = "squid_game_review.csv"
f = open(filename, "w", newline="", encoding='UTF-8')
writer = csv.writer(f)
writer.writerow(['reviews'])

for review in reviews:
    text = review.text
    print(text)
    text.replace('\n', '')
    writer.writerow([text])
    


 Great show, horrible VIPs

 The VIP's are dumb & ridiculous.

 Loved it, but...

 Yes, the VIPs sucked, but that didn't ruin the series

 Atrocious translation

 Definitely one of the most well-made Netflix shows by Korea

 Best description? Uncomfortably genius! And quite disturbing!

 A realistic version of Alice in Borderland

 Hollywood is done

 Terrible Subtitle

 Bingeworthy

 The squid financial game

 Parasite + Alice in Borderland

 Korean Black Mirror

 Fun with some disappointing bits (SPOILERS)

 Great show but wow the acting gets bad in episode 7

 mindblowing

 I Really hope there's a second season and a third

 It was good...for a while

 Spine chilling game

 BINGE WORTHY

 INTERESTING and Got Me Hooked

 I really love it

 Great show!!

 A diamond in the rough from Korea.

 Fantastic.

 It could have been a solid 8.

 Gi-Hun is played by Lee Jung-Jae

 Great start but dissapointed ending

 Enjoyable and addictive but.....!

 Extremely good dark action thriller except

In [9]:
import pandas as pd

data=pd.read_csv('./squid_game_review.csv', sep="\n")
data.head()

,reviews
0,"Great show, horrible VIPs\n"
1,The VIP's are dumb & ridiculous.\n
2,"Loved it, but...\n"
3,"Yes, the VIPs sucked, but that didn't ruin th..."
4,Atrocious translation\n


In [10]:
type(data)

pandas.core.frame.DataFrame

In [11]:
data.iloc[1]

reviews     The VIP's are dumb & ridiculous.\n
Name: 1, dtype: object

In [12]:
data['clean_doc'] = data['reviews'].str.replace("[^a-zA-Z]"," ", regex=True)
data['clean_doc'] = data['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data['clean_doc'] = data['clean_doc'].apply(lambda x: x.lower())

In [13]:
data['clean_doc'].head()

0             great show horrible vips
1                      dumb ridiculous
2                                loved
3    vips sucked that didn ruin series
4                atrocious translation
Name: clean_doc, dtype: object

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = data['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc

0                  [great, show, horrible, vips]
1                             [dumb, ridiculous]
2                                        [loved]
3                   [vips, sucked, ruin, series]
4                       [atrocious, translation]
                         ...                    
520                         [squid, masterpiece]
521                          [excellent, series]
522                                      [loved]
523                    [wonderful, loved, every]
524    [captivating, gory, critique, capitalism]
Name: clean_doc, Length: 525, dtype: object

In [15]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(4, 1), (5, 1)]


In [16]:
print(dictionary[4])

dumb


In [17]:
len(dictionary)

602

In [18]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.104*"episode" + 0.069*"loved" + 0.069*"awesome" + 0.046*"last"')
(1, '0.121*"amazing" + 0.048*"series" + 0.035*"great" + 0.029*"netflix"')
(2, '0.106*"korea" + 0.036*"bingeworthy" + 0.036*"done" + 0.026*"show"')
(3, '0.220*"great" + 0.149*"show" + 0.062*"series" + 0.047*"good"')
(4, '0.047*"meets" + 0.024*"series" + 0.024*"wish" + 0.024*"main"')
(5, '0.082*"fantastic" + 0.035*"well" + 0.024*"korean" + 0.015*"made"')
(6, '0.086*"borderland" + 0.086*"alice" + 0.039*"show" + 0.025*"violent"')
(7, '0.057*"movie" + 0.047*"good" + 0.046*"predictable" + 0.023*"surprising"')
(8, '0.086*"korean" + 0.062*"love" + 0.038*"hell" + 0.033*"first"')
(9, '0.085*"binge" + 0.085*"worth" + 0.057*"watching" + 0.057*"worthy"')
(10, '0.140*"best" + 0.067*"show" + 0.037*"korean" + 0.034*"time"')
(11, '0.123*"overrated" + 0.084*"excellent" + 0.028*"enjoyable" + 0.028*"could"')
(12, '0.044*"hooked" + 0.033*"interesting" + 0.022*"story" + 0.017*"seen"')
(13, '0.151*"watch" + 0.059*"series" + 0.057*"must" 

In [19]:
import pyLDAvis.gensim_models
import pyLDAvis


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\LOVE.DESKTOP-5CS2DDB\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [20]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)
    print()

0 번째 문서의 topic 비율은 [(0, 0.010000249), (1, 0.010000249), (2, 0.010000249), (3, 0.59861076), (4, 0.010000249), (5, 0.010000249), (6, 0.22138472), (7, 0.010000249), (8, 0.010000249), (9, 0.010000249), (10, 0.010000249), (11, 0.010000249), (12, 0.010000249), (13, 0.010000249), (14, 0.010000249), (15, 0.010000249), (16, 0.010000249), (17, 0.010000249), (18, 0.010000249), (19, 0.010000249)]

1 번째 문서의 topic 비율은 [(0, 0.016666992), (1, 0.016666992), (2, 0.016666992), (3, 0.016666992), (4, 0.016666992), (5, 0.016666992), (6, 0.016666992), (7, 0.016666992), (8, 0.016666992), (9, 0.016666992), (10, 0.016666992), (11, 0.016666992), (12, 0.016666992), (13, 0.016666992), (14, 0.016666992), (15, 0.016666992), (16, 0.016666992), (17, 0.016666992), (18, 0.016666992), (19, 0.68332714)]

2 번째 문서의 topic 비율은 [(0, 0.52499884), (1, 0.02500006), (2, 0.02500006), (3, 0.02500006), (4, 0.02500006), (5, 0.02500006), (6, 0.02500006), (7, 0.02500006), (8, 0.02500006), (9, 0.02500006), (10, 0.02500006), (11, 0.025000

In [21]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [22]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,3.0,0.5986,"[(0, 0.010000249), (1, 0.010000249), (2, 0.010..."
1,1,19.0,0.6833,"[(0, 0.01666699), (1, 0.01666699), (2, 0.01666..."
2,2,0.0,0.5250,"[(0, 0.52499884), (1, 0.025000058), (2, 0.0250..."
3,3,3.0,0.8100,"[(0, 0.010000182), (1, 0.010000182), (2, 0.010..."
4,4,17.0,0.6833,"[(0, 0.016666882), (1, 0.016666882), (2, 0.016..."
5,5,10.0,0.5410,"[(2, 0.15000038), (5, 0.1875812), (10, 0.54098..."
6,6,10.0,0.8643,"[(10, 0.86428225)]"
7,7,6.0,0.4100,"[(0, 0.010000176), (1, 0.010000176), (2, 0.010..."
8,8,2.0,0.6833,"[(0, 0.016666759), (1, 0.016666759), (2, 0.683..."
9,9,7.0,0.6833,"[(0, 0.016666971), (1, 0.016666971), (2, 0.016..."
